# Down-scaled multi-area model

### Create config file

In [ ]:
with open('config.py', 'w') as fp:
    fp.write(
'''import os
base_path = os.path.abspath(".")
data_path = os.path.abspath("simulations")
jobscript_template = "python {base_path}/run_simulation.py {label}"
submit_cmd = "bash -c"
''')

### Import dependencies

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import nest

from multiarea_model import MultiAreaModel
from config import base_path

In [ ]:
!pip install nested_dict dicthash

### Jupyter notebook display format setting

In [ ]:
# specify the format the table in output
%%html
<style>
table {float:left}
</style>

<br>

## Specify paramters of model

### 1. Scaling factor (scale_down_to)
**Scaling factor** (scale_down_to) is the parameter which defines the the ratio of the full scale multi-area model being down-scaled to a model with fewer neurons and indegrees so as to be simulated on machines with lower computational ability and the simulation results can be obtained within relative shorter period of time.<br> <br> 
Neurons and indegrees are both scaled down to 0.5%, where the model can usually be simulated on a local machine.<br> **Warning**: This will not yield reasonable dynamical results from the network and is only meant to demonstrate the simulation workflow.**

|Parameter      |Parameter description  |Variable                     |Value               |Value description  |
|:-------------:|:----------------------|:---------------------------:|:------------------:|:------------------|
|Scaling factor |                       | scale_down_to               |0.005               |                   |

In [ ]:
scale_down_to = 0.005 # Change it to 1. for running the fullscale network

### 2. Model and simulation parameters

#### 2.1 Connection parameters (conn_params)

| Parameter | Parameter description | Variable                    | Value              | Value description |
|:---------:|:----------------------|:---------------------------:|:------------------:|:------------------|
|           |                       | replace_non_simulated_areas | 'het_poisson_stat' |                   |
|           |                       | g                           | -11.               |                   |
|           |                       | K_stable                    | 'K_stable.npy'     |                   |
|           |                       | fac_nu_ext_TH               | 1.2                |                   |
|           |                       | fac_nu_ext_5E               | 1.125              |                   |
|           |                       | fac_nu_ext_6E               | 1.41666667         |                   |
|           |                       | av_indegree_V1              | 3950.              |                   |

In [ ]:
conn_params = {'replace_non_simulated_areas': 'het_poisson_stat',
               'g': -11.,
               'K_stable': 'K_stable.npy',
               'fac_nu_ext_TH': 1.2,
               'fac_nu_ext_5E': 1.125,
               'fac_nu_ext_6E': 1.41666667,
               'av_indegree_V1': 3950.}

#### 2.2 Input parameters (input_params)

| Parameter | Parameter description | Variable                      | Value                | Value description |
|:---------:|:----------------------|:-----------------------------:|:--------------------:|:------------------|
|           |                       | rate_ext                      |      10.             |                   |

In [ ]:
input_params = {'rate_ext': 10.}

#### 2.3 Neuron parameters (neuron_params)

| Parameter | Parameter description | Variable                    | Value              | Value description |
|:---------:|:----------------------|:---------------------------:|:------------------:|:------------------|
|           |                       | V0_mean                     | -150.              |                   |
|           |                       | V0_sd                       | 50.                |                   |

In [ ]:
neuron_params = {'V0_mean': -150.,
                 'V0_sd': 50.}

#### 2.4 Network parameters (network_params)

| Parameter                               | Parameter description | Variable              | Value                         | Value description |
|:---------------------------------------:|:----------------------|:---------------------:|:-----------------------------:|:------------------|
| Scaling factor of the number of neurons |                       | N_scaling             | scale_down_to                 |                   |
| Scaling factor of the number of synapses|                       | K_scaling             | scale_down_to                 |                   |
| Fullscale rates                         |                       | fullscale_rates       | 'tests/fullscale_rates.json'  |                   |
| Input parameters                        |                       | input_params          | input_params                  |                   |
| Connections parameters                  |                       | connection_params     | conn_params                   |                   |
| Neuron parameters                       |                       | neuron_params         | neuron_params                 |                   |

In [ ]:
network_params = {'N_scaling': scale_down_to,
                  'K_scaling': scale_down_to,
                  'fullscale_rates': 'tests/fullscale_rates.json',
                  'input_params': input_params,
                  'connection_params': conn_params,
                  'neuron_params': neuron_params}

### 2.5 Simulation paramters (sim_params)

| Parameter             | Parameter description | Variable             | Value              | Value description |
|:---------------------:|:----------------------|:--------------------:|:------------------:|:------------------|
|Simulation time        |                       |t_sim                 |2000.               |                   |
|Number of processes    |                       |num_processes         |1                   |                   |
|Number of local threads|                       |local_num_threads     |1                   |                   |
|                       |                       |recording_dict        |input_params        |                   |
|                       |                       |record_vm             |False               |                   |

In [ ]:
sim_params = {'t_sim': 2000.,
              'num_processes': 1,
              'local_num_threads': 1,
              'recording_dict': {'record_vm': False},
              'rng_seed': 1} # global random seed


### 2.6. Theory paramters (theory_params)

| Parameter | Parameter description | Variable              | Value                         | Value description |
|:---------:|:----------------------|:---------------------:|:-----------------------------:|:------------------|
|           |                       | dt                    | 0.1                           |                   |

In [ ]:
theory_params = {'dt': 0.1}

<br>

## Instantiate a multi-area model and analyse

### 1. Insantiate a multi-area model 

In [ ]:
M = MultiAreaModel(network_params, simulation=True,
                   sim_spec=sim_params,
                   theory=True,
                   theory_spec=theory_params)

### 2. Predict firing rates from theory

In [ ]:
p, r = M.theory.integrate_siegert()
print("Mean-field theory predicts an average "
      "rate of {0:.3f} spikes/s across all populations.".format(np.mean(r[:, -1])))

### 3. Extract connectivity

The connectivity and neuron numbers are stored in the attributes of the model class. Neuron numbers are stored in `M.N` as a dictionary (and in `M.N_vec` as an array), indegrees in `M.K` as a dictionary (and in `M.K_matrix` as an array). Number of synapses can also be access via `M.synapses` (and in `M.syn_matrix` as an array).

#### 3.1 Node indegrees

In [ ]:
# Dictionary of nodes indegrees organized as:
# {<source_area>: {<source_pop>: {<target_area>: {<target_pop>: indegree_values}}}}
M.K

#### 3.2 Synapses

In [ ]:
# Dictionary of synapses that target neurons receive, it is organized as:
# {<source_area>: {<source_pop>: {<target_area>: {<target_pop>: number_of_synapses}}}}
M.synapses

<br>

## Run the simulation

In [ ]:
# run the simulation, depending on the model parameter and downscale ratio, the running time varies largely.
M.simulation.simulate()

<br>

## Simulation results analysis

The following instructions will work when the `simulate` parameter is set to `True` during the creation of the MultiAreaModel object, and the `M.simulation.simulate()` method is executed.

In [ ]:
# Uncomment the lines in this code cell below to test if the number of synapses created by NEST matches the expected values

# """
# Test if the correct number of synapses has been created.
# """
# print("Testing synapse numbers")
# for target_area_name in M.area_list:
#     target_area = M.simulation.areas[M.simulation.areas.index(target_area_name)]
#     for source_area_name in M.area_list:
#         source_area = M.simulation.areas[M.simulation.areas.index(source_area_name)]
#         for target_pop in M.structure[target_area.name]:
#             target_nodes = target_area.gids[target_pop]
#             for source_pop in M.structure[source_area.name]:
#                 source_nodes = source_area.gids[source_pop]
#                 created_syn = nest.GetConnections(source=source_nodes,
#                                                   target=target_nodes)
#                 syn = M.synapses[target_area.name][target_pop][source_area.name][source_pop]
#                 assert(len(created_syn) == int(syn))

To obtain the connections information, you can extract the lists of connected sources and targets. Moreover, you can access additional synaptic details, such as synaptic weights and delays.

In [ ]:
conns = nest.GetConnections()
conns_sparse_matrix = conns.get(['source', 'target', 'weight'])

srcs = conns_sparse_matrix['source']
tgts = conns_sparse_matrix['target']
weights = conns_sparse_matrix['weight']

You can determine the area and subpopulation to which the neuron ID ranges belong by referring to the file `network_gids.txt`, which is automatically generated during network creation.

In [ ]:
# Open the file using a with statement
with open(os.path.join(M.simulation.data_dir,"recordings/network_gids.txt"), "r") as file:
    # Read the contents of the file
    gids = file.read()

# Print the contents
print(gids)

<br>

## Load and process data of simulation results

### 1. Load spike data

In [ ]:
data = np.loadtxt(M.simulation.data_dir + '/recordings/' + M.simulation.label + "-spikes-1-0.dat", skiprows=3)

### 2. Compute instantaneous rate per neuron across all populations

In [ ]:
tsteps, spikecount = np.unique(data[:,1], return_counts=True)
rate = spikecount / M.simulation.params['dt'] * 1e3 / np.sum(M.N_vec)

<br>

## Simulation results visualization

### 1. Instantaneous and mean rate

In [ ]:
fig, ax = plt.subplots()
ax.plot(tsteps, rate)
ax.plot(tsteps, np.average(rate)*np.ones(len(tsteps)), label='mean')
ax.set_title('instantaneous rate across all populations')
ax.set_xlabel('time (ms)')
ax.set_ylabel('rate (spikes / s)')
ax.set_xlim(0, sim_params['t_sim'])
ax.set_ylim(0, 50)
ax.legend()